Universidade Federal de Pernambuco - UFPE \
Centro de Informática - CIn \
Ciência da computação - Pós graduação \
Deep Learn 2024.1 \
Student: Giovanna Ily Farias Ramalho

#Robo do processo

##utilizando técnicas de reconhecimento de entidades nomeadas para treinamento de Inteligencia Artificial para a area juridica

# Import

In [ ]:
# Instalar a última versão do spaCy
!pip install --upgrade spacy

# Baixar e instalar o modelo pt_core_news_lg da versão 3.5.0 (versão mais recente compatível até agora)
!pip install https://github.com/explosion/spacy-models/releases/download/pt_core_news_lg-3.5.0/pt_core_news_lg-3.5.0.tar.gz

# Instalar a biblioteca do modelo pt_core_news_lg
!pip install pt_core_news_lg

# Instalar a biblioteca tqdm para barras de progresso
!pip install tqdm

# Instalar pandas para manipulação de CSV
!pip install pandas

# Instalar TensorFlow
!pip install tensorflow

# Instalar Hugging Face Transformers e Datasets
!pip install transformers datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.0 MB/s eta 0:00:00
  Created wheel for pt-core-news-lg: filename=pt_core_news_lg-3.5.0-py3-none-any.whl size=568212748 sha256=82dcfb0717c989266ddbef7bba758e5f243bb11fa3cdb4f79a2afb0c5f7fee3d
  Stored in directory: /root/.cache/pip/wheels/a7/71/d8/b57804a203726cd0c1a9ce7d456be03ea45f5d2e3a1abdbd7d
Successfully built pt-core-news-lg
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Un

In [ ]:
# Importar bibliotecas necessárias
import spacy
import nltk
import re
import spacy.cli
import csv
import json
from bs4 import BeautifulSoup as bs
from nltk.corpus import stopwords
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import torch
import gc
from torch.utils.data import TensorDataset
from datasets import Dataset

# Importar bibliotecas do Hugging Face
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

# Baixar os recursos necessários do NLTK
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Conectar com google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Caminho para o arquivo CSV
csv_path = '/content/drive/MyDrive/legal_dataset/train.csv'

# Carregar os dados do arquivo CSV corrigindo a delimitação das colunas
df = pd.read_csv(csv_path, delimiter=',', encoding='utf-8')
df.columns = ['id', 'tokens', 'ner_tags']

# Ler o CSV em um DataFrame
df = pd.read_csv(csv_path)

# Exibir as primeiras linhas do DataFrame para verificação
print(df.head())

Mounted at /content/drive
   id                                             tokens  \
0   0  ['EMENTA' ':' 'APELAÇÃO' 'CÍVEL' '-' 'AÇÃO' 'D...   
1   1  ['-' 'O' 'art' '.' '178' ',' 'II' ',' 'do' 'CP...   
2   2  ['-' 'Tratando-se' 'de' 'ação' 'indenizatória'...   
3   3  ['-' 'Tendo' 'o' 'vício' 'sido' 'arguido' 'pel...   
4   4  ['-' 'Preliminar' 'acolhida' 'para' 'reconhece...   

                                            ner_tags  
0  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0...  
1  [ 0  0  9 10 10 10 10 10 10 10  0  0  0  0  1 ...  
2  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0 0...  
3  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...  
4                              [0 0 0 0 0 0 0 0 0 0]  


In [ ]:
# Carregar os dados do arquivo CSV corrigindo a delimitação das colunas
df = pd.read_csv(csv_path, delimiter=',', encoding='utf-8')
df.columns = ['id', 'tokens', 'ner_tags']

# Converter todos os valores da coluna 'text' para string e tratar valores nulos
df['tokens'] = df['tokens'].astype(str).fillna('')

# Garantir que a coluna de etiquetas também esteja no formato correto
df['ner_tags'] = df['ner_tags'].astype(str).fillna('')

#Preparar os Dados para Tokenização

In [ ]:
# Preparar os dados para tokenização
data = []
for index, row in df.iterrows():
    tokens = row['tokens'].split()
    tags = row['ner_tags'].split()
    data.append({"tokens": tokens, "tags": tags})

# Converter para DataFrame
df_tokens = pd.DataFrame(data)

# Verificar rótulos únicos para atualização do label_list
unique_labels = set([label for labels in df_tokens['tags'] for label in labels])
print("Rótulos únicos:", unique_labels)

# Atualizar label_list com os rótulos reais
label_list = list(unique_labels)
print("Lista de rótulos atualizada:", label_list)

# Verificar comprimento das frases
df_tokens['length'] = df_tokens['tokens'].apply(len)
print(df_tokens['length'].describe())

# Truncar frases com comprimento maior que o máximo permitido
max_length = 128
df_tokens['tokens'] = df_tokens['tokens'].apply(lambda x: x[:max_length])
df_tokens['tags'] = df_tokens['tags'].apply(lambda x: x[:max_length])


Rótulos únicos: {'4', '10', '9', '[1', '[7', '4]', '[3', '8', '[11', '0]', '12', '7', '6', '1', '5]', '5', '2', '11', '[9', '[', '[0]', '[]', '[0', '12]', '0', '3'}
Lista de rótulos atualizada: ['4', '10', '9', '[1', '[7', '4]', '[3', '8', '[11', '0]', '12', '7', '6', '1', '5]', '5', '2', '11', '[9', '[', '[0]', '[]', '[0', '12]', '0', '3']
count    7828.000000
mean       29.289474
std        34.722588
min         1.000000
25%         6.000000
50%        20.000000
75%        41.000000
max       755.000000
Name: length, dtype: float64


#Tokenização

In [ ]:
# Carregar o tokenizer do BERTimbau
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

# Função para tokenizar e alinhar os labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=max_length
    )

    labels = []
    for i, label in enumerate(examples["tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx and word_idx < len(label):
                label_ids.append(label_list.index(label[word_idx]))
            elif word_idx < len(label):
                label_ids.append(label_list.index(label[word_idx]))
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Converter DataFrame para Dataset do Hugging Face
dataset = Dataset.from_pandas(df_tokens)

In [ ]:
# Tokenizar e alinhar labels
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/7828 [00:00<?, ? examples/s]

In [ ]:
# Definir Data Collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Converter para tensores PyTorch
inputs = torch.tensor(tokenized_dataset["input_ids"])
attention_masks = torch.tensor(tokenized_dataset["attention_mask"])
labels = torch.tensor(tokenized_dataset["labels"])

In [ ]:
# Dataset final
dataset = TensorDataset(inputs, attention_masks, labels)

In [ ]:
# Definir argumentos de treinamento com um menor batch size
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Carregar modelo pré-treinado para classificação de tokens
model = AutoModelForTokenClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=len(label_list)
)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Treinamento

In [ ]:
# Treinador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [ ]:
# Treinar o modelo
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.145900,0.070816
2,0.068100,0.038768
3,0.040800,0.028358


TrainOutput(global_step=5871, training_loss=0.11974104639416232, metrics={'train_runtime': 11914.9039, 'train_samples_per_second': 1.971, 'train_steps_per_second': 0.493, 'total_flos': 1534406125897728.0, 'train_loss': 0.11974104639416232, 'epoch': 3.0})

# Resultados do Treinamento

Após três épocas de treinamento, observei uma melhora constante nas métricas de perda tanto para os dados de treinamento quanto para os de validação. Aqui estão os resultados detalhados:

| Epoch | Training Loss | Validation Loss |
|-------|---------------|-----------------|
| 1     | 0.145900      | 0.070816        |
| 2     | 0.068100      | 0.038768        |
| 3     | 0.040800      | 0.028358        |

A perda de treinamento diminuiu consistentemente, mostrando que o modelo estava aprendendo bem a partir dos dados. Mais importante ainda, a perda de validação também apresentou uma redução significativa, indicando que o modelo estava generalizando bem para dados não vistos. A diferença mínima entre as perdas de treinamento e validação sugere que o modelo não estava sofrendo de overfitting.

# Reflexões Finais

O treinamento do modelo foi uma experiência gratificante. O tempo total de treinamento foi de cerca de 3 horas e 18 minutos, um investimento que se mostrou valioso ao ver as melhorias nas métricas de perda. Com uma média de 1.971 amostras processadas por segundo e 0.493 passos de treinamento por segundo, o processo foi eficiente.

Sinto que há ainda espaço para melhorias. Ajustar os hiperparâmetros e explorar técnicas como validação cruzada pode ajudar a refinar ainda mais o modelo. Além disso, calcular métricas adicionais como precisão, recall e F1-score será crucial para uma avaliação mais completa da performance do modelo.